### Prepare the data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

In [3]:
PATH_TO_DATA = Path('/Users/gjke/Documents/uni/faktual/swissdata')

In [4]:
PATH_TO_MODEL = Path('/Users/gjke/Documents/uni/faktual/transformer-lm/de345-root')

#### Reformat train.csv using <tldr\> token

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv(PATH_TO_DATA / 'data_train.csv')

In [7]:
train_df.head()

,source,summary
0,Minghella war der Sohn italienisch-schottische...,"Anthony Minghella, CBE war ein britischer Film..."
1,Ende der 1940er Jahre wurde eine erste Auteur-...,Die Auteur-Theorie ist eine Filmtheorie und di...
2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","Alfredo James ""Al"" Pacino ist ein US-amerikani..."
3,Der Name der Alkalimetalle leitet sich von dem...,Als Alkalimetalle werden die chemischen Elemen...
4,Die Arbeit ist bereits seit dem Altertum Gegen...,Das deutsche Arbeitsrecht ist ein Rechtsgebiet...


In [8]:
train_df.shape

(100000, 2)

In [9]:
TLDR = '<tldr>'

In [10]:
if not os.path.exists(PATH_TO_DATA / 'train'):
    os.mkdir(PATH_TO_DATA / 'train')
with open(PATH_TO_DATA / 'train/data_train_tldr.txt', 'w') as f:
    for row in train_df.loc[:int(train_df.shape[0]*0.85)].itertuples(index=False):
        f.write(row[0] + TLDR + row[1] + "\n")

In [11]:
if not os.path.exists(PATH_TO_DATA / 'valid'):
    os.mkdir(PATH_TO_DATA / 'valid')
with open(PATH_TO_DATA / 'valid/data_val_tldr.txt', 'w') as f:
    for row in train_df.loc[int(train_df.shape[0]*0.85):int(train_df.shape[0]*0.95)].itertuples(index=False):
        f.write(row[0] + TLDR + row[1] + "\n")

In [12]:
if not os.path.exists(PATH_TO_DATA / 'test'):
    os.mkdir(PATH_TO_DATA / 'test')
with open(PATH_TO_DATA / 'test/data_test_tldr.txt', 'w') as f:
    for row in train_df.loc[int(train_df.shape[0]*0.95):].itertuples(index=False):
        f.write(row[0] + TLDR + row[1] + "\n")

In [13]:
del train_df

### Encode the data

In [14]:
!sp-encode $PATH_TO_DATA $PATH_TO_MODEL/sp.model $PATH_TO_DATA/encoded

Reading corpora: ['/Users/gjke/Documents/uni/faktual/swissdata']
/Users/gjke/Documents/uni/faktual/swissdata/train: 100%|█| 1/1 [05:09<00:00, 309.16s/it]
/Users/gjke/Documents/uni/faktual/swissdata/valid: 100%|█| 1/1 [00:30<00:00, 30.25s/it]
/Users/gjke/Documents/uni/faktual/swissdata/test: 100%|█| 1/1 [00:15<00:00, 15.55s/it]
Saving encoded split train to /Users/gjke/Documents/uni/faktual/swissdata/encoded/train.npy
Saving encoded split valid to /Users/gjke/Documents/uni/faktual/swissdata/encoded/valid.npy
Saving encoded split test to /Users/gjke/Documents/uni/faktual/swissdata/encoded/test.npy


### Start training 

In [15]:
!gpt-2 $PATH_TO_MODEL $PATH_TO_DATA/encoded $PATH_TO_MODEL/sp.model --n_embed=1024 --n_ctx=1024 --n_head=16 --n_layer=24 --n_hidden=1024 --batch_size=3 --gradient_checkpointing=0 --save_every=5000

{
    "argv": "/Users/gjke/opt/anaconda3/envs/german-gpt2/bin/gpt-2 /Users/gjke/Documents/uni/faktual/transformer-lm/de345-root /Users/gjke/Documents/uni/faktual/swissdata/encoded /Users/gjke/Documents/uni/faktual/transformer-lm/de345-root/sp.model --n_embed=1024 --n_ctx=1024 --n_head=16 --n_layer=24 --n_hidden=1024 --batch_size=3 --gradient_checkpointing=0 --save_every=5000",
    "batch_size": 3,
    "epochs": 10,
    "g_accum_gradients": 1,
    "hparams": {
        "gradient_checkpointing": 0,
        "n_ctx": 1024,
        "n_embed": 1024,
        "n_head": 16,
        "n_hidden": 1024,
        "n_layer": 24,
        "n_vocab": 50000
    },
    "lr": 0.00025
}
Loading dataset from /Users/gjke/Documents/uni/faktual/swissdata/encoded
Train dataset has 76,796,624 tokens
Validation dataset has 7,831,779 tokens
Resuming from seen_tokens 11,962,060,800
epochs:   0%|                                            | 0/10 [00:00<?, ?it/s]
epochs: 155it [00:00, 944937.67it/s]

validation:   0%|  